# Entanglement Classifier Training Trial

## Importing Dependencies

In [70]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sympy.physics.quantum import TensorProduct

from components.system import System
from components.connection import ConnectionFactory, Connection
from components.model import Model
from components.connection_functions import DrivenCascadeFunction, EnergyExchangeFunction
from components.state_generator import generateMixedBatch, generatePureBatch, generateMixedBatchUsingPure

## Generating Training Data

In [71]:
samples = 2000
test_size = 0.5

In [72]:
x_pure = generatePureBatch(int(samples/2))
y_pure = np.ones(int(samples/2))
x_mixed = generateMixedBatchUsingPure(int(samples/2))
y_mixed = np.zeros(int(samples/2))

In [73]:
x = np.concatenate([x_pure, x_mixed])
y = np.concatenate([y_pure, y_mixed])

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42, shuffle=True)

## Generate Reservoir Measurements

### Setting up the system

In [75]:
reservoir_nodes=4
system_nodes=1

In [76]:
"""
Defining System setup
"""

system_state = np.array([[1, 0], [0, 0]])
system_node_list = [0]

if len(system_node_list) != system_nodes:
    raise Exception

system_connections = {}

system = System(
    init_quantum_state=system_state, nodes=system_node_list, connections=system_connections
)

### Setting up the interface

In [77]:
"""
Defining Interface setup
"""

interfaceFactory = ConnectionFactory(DrivenCascadeFunction, gamma_1=[8,12], gamma_2=[8,12])

### Setting up the reservoir

In [78]:
"""
Defining Reservoir setup
"""

reservoirFactory = ConnectionFactory(EnergyExchangeFunction, is_hamiltonian=True, J=[0.5,1.5])

### Setting up the model

In [79]:
model = Model()
model.setSystem(system)
model.setReservoirConnectionFac(reservoirFactory)
model.setInterfaceConectionFac(interfaceFactory)
model.generateReservoir(reservoir_nodes, init_quantum_state=0)
model.generateInterface(connection_rate=.5)
model.setRunDuration(0.1)
model.setRunResolution(0.01)

### Running the model

In [80]:
d_train = []
d_test = []
measurement_iteration = -1

for state in x_train:
    model.system.init_quantum_state = state
    model.run()
    d_train.append(np.real(list(model.modelLog.excitation_time_log.values())[measurement_iteration]))

for state in x_test:
    model.system.init_quantum_state = state
    model.run()
    d_test.append(np.real(list(model.modelLog.excitation_time_log.values())[measurement_iteration]))

## Classical Processing

In [81]:
sgd = SGDClassifier()
sgd.fit(d_train, y_train)

SGDClassifier()

In [82]:
sgd.score(d_test, y_test)

0.728

In [83]:
from sklearn.metrics import confusion_matrix

In [84]:
wrong = []

for i in range(len(y_test)):
    if y_test[i] != y_pred[i]:
        wrong.append([x_test[i], np.trace(np.matmul(x_test[i], x_test[i])), y_test[i]])

In [85]:
wrong[10]

[array([[0.39946905, 0.48978927],
        [0.48978927, 0.60053095]]),
 1.0000000000000004,
 1.0]